In [1]:
from pathlib import Path
import os
import numpy as np
import pandas as pd
import io
import sys
import logging
from new_strategy import TradingStrategy, Asset, BetSizingMethod, get_bet_sizing
from meta_strategy import MetaLabelingStrategy
import nbimporter
from backtest import Backtest
from add_features import generate_meta_labeled_data, merge_with_raw_features
from metalabel_backtest import MetaModelHandler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
%load_ext autoreload
%autoreload 2

In [2]:
def split_price_data(price_data: pd.DataFrame, split_ratio: float = 0.7):
    split_idx = int(len(price_data) * split_ratio)
    return price_data.iloc[:split_idx], price_data.iloc[split_idx:]

def run_training_phase(asset: Asset, method: BetSizingMethod, price_data: pd.DataFrame, feature_cols: list):
    # Run base strategy
    past_returns = price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)
    
    strategy = TradingStrategy(price_data, asset.value, bet_sizing, method)
    strategy.generate_signals()
    strategy.simulate_trades()
    
    trade_df = strategy.get_trade_data()
    labeled = generate_meta_labeled_data(trade_df)
    #labeled_with_features = merge_with_raw_features(labeled, asset.value)
    labeled_with_features = labeled
    
    print(labeled_with_features.columns)
    return labeled_with_features

def train_meta_model(train_df: pd.DataFrame, feature_cols: list) -> MetaModelHandler:
    model = MetaModelHandler()
    model.train(train_df, feature_cols)
    return model

def run_parallel_evaluation(asset, method, test_price_data, signals, meta_model, feature_cols):
    past_returns = test_price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)

    #  First, initialize Meta strategy to access feature-enriched data
    meta_strategy = MetaLabelingStrategy(
        test_price_data.copy(), asset.value, bet_sizing, method,
        meta_model_handler=meta_model
    )
    feature_data = meta_strategy.data.copy()

    #  Use feature_data for both strategies
    baseline = TradingStrategy(feature_data.copy(), asset.value, bet_sizing, method)
    baseline.trade_signals = signals

    filtered = MetaLabelingStrategy(feature_data.copy(), asset.value, bet_sizing, method, meta_model_handler=meta_model,feature_cols=feature_cols)
    filtered.trade_signals = signals

    print("\n📊 Baseline features in .data:")
    print(baseline.data.columns.tolist())

    print("\n📊 Filtered features in .data:")
    print(filtered.data.columns.tolist())
    print("\n🕒 Baseline index:", baseline.data.index.name, baseline.data.index.dtype)
    print("🕒 Filtered index:", filtered.data.index.name, filtered.data.index.dtype)

    baseline.simulate_trades()
    filtered.simulate_trades()

    return baseline, filtered


def compare_backtests(baseline: TradingStrategy, filtered: MetaLabelingStrategy, asset: Asset, method: BetSizingMethod):
    print("\n[BASELINE]")
    baseline_bt = Backtest(baseline)
    baseline_bt.run_analysis()
    baseline_buffer = io.StringIO()
    sys.stdout = baseline_buffer
    baseline_bt.print_summary()
    sys.stdout = sys.__stdout__
    baseline_summary = baseline_buffer.getvalue()
    print(baseline_summary)

    print("\n[META-FILTERED]")
    filtered_bt = Backtest(filtered)
    filtered_bt.run_analysis()
    filtered_buffer = io.StringIO()
    sys.stdout = filtered_buffer
    filtered_bt.print_summary()
    sys.stdout = sys.__stdout__
    filtered_summary = filtered_buffer.getvalue()
    print(filtered_summary)

    print(f"\nTrades Rejected by Meta-Model: {filtered.rejected_trades}")

    output_dir = "results_metalabel"
    os.makedirs(output_dir, exist_ok=True)

    asset_name = asset.value.lower()
    method_name = method.value.lower()

    baseline_csv = f"baseline_{asset_name}_{method_name}.csv"
    filtered_csv = f"filtered_{asset_name}_{method_name}.csv"
    baseline_txt = f"baseline_{asset_name}_{method_name}.txt"
    filtered_txt = f"filtered_{asset_name}_{method_name}.txt"

    baseline.get_trade_data().to_csv(os.path.join(output_dir, baseline_csv), index=False)
    filtered.get_trade_data().to_csv(os.path.join(output_dir, filtered_csv), index=False)

    with open(os.path.join(output_dir, baseline_txt), "w") as f:
        f.write(baseline_summary)

    with open(os.path.join(output_dir, filtered_txt), "w") as f:
        f.write(filtered_summary)

    print(f"\n✅ CSVs saved to: {output_dir}")
    print(f" - {baseline_csv}")
    print(f" - {filtered_csv}")
    print(f"\n✅ Summaries saved to:")
    print(f" - {baseline_txt}")
    print(f" - {filtered_txt}")


if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)

    asset = Asset.XAUUSD
    method = BetSizingMethod.FIXED

    feature_cols = ["attempt", "ma_14", "min_price_30", "max_price_30","atr_14"]
    price_path = Path(f"data/processed/{asset.value}/combined_data.csv")
    price_data = pd.read_csv(price_path, index_col='timestamp', parse_dates=True)

    # Split
    train_data, test_data = split_price_data(price_data)

    # Training
    train_trades = run_training_phase(asset, method, train_data, feature_cols)
    
    #clean_features = train_trades[feature_cols].replace([np.inf, -np.inf], np.nan).dropna()
    #train_trades_cleaned = train_trades.loc[clean_features.index]
    train_trades_cleaned = train_trades.replace([np.inf, -np.inf], np.nan).dropna(subset=feature_cols)

    train_trades_cleaned.to_csv("results_metalabel/train_data_debug.csv", index=False)
    print("✅ Training data saved to results_metalabel/train_data_debug.csv")

    meta_model = train_meta_model(train_trades_cleaned, feature_cols)

    # Reuse signals
    signal_gen = TradingStrategy(test_data.copy(), asset.value, get_bet_sizing(method), method)
    signal_gen.generate_signals()
    shared_signals = signal_gen.trade_signals

    print("\n🔍 Entry Time Check (from shared_signals dict):")
    print("Type:", type(shared_signals))
    print("Keys in shared_signals:", list(shared_signals.keys()))

    # Try to peek at the first signal (depending on structure)
    first_key = next(iter(shared_signals))
    first_signal_list = shared_signals[first_key]
    first_signal = first_signal_list[0]  # Get the first signal from the list
    print("Type of entry_time:", type(first_signal['entry_time']))

    # Evaluation
    base_strat, filtered_strat = run_parallel_evaluation(
        asset, method, test_data, shared_signals, meta_model, feature_cols
    )

    # Compare
    compare_backtests(base_strat, filtered_strat, asset, method)

2025-06-17 11:23:50,374 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing


📦 Creating TradeSetup with:
  entry_time: 2020-01-02 00:00:00+00:00
  atr_14: nan, ma_14: nan, min_price_30: nan, max_price_30: nan
  context keys: ['ma_14', 'min_price_30', 'max_price_30', 'atr_14', 'attempt', 'ref_close', 'duration_minutes', 'session', 'eval_f1', 'eval_accuracy', 'eval_precision', 'eval_recall', 'n_total_seen', 'n_window_obs', 'session_code']
📦 Creating TradeSetup with:
  entry_time: 2020-01-02 08:00:00+00:00
  atr_14: nan, ma_14: nan, min_price_30: nan, max_price_30: nan
  context keys: ['ma_14', 'min_price_30', 'max_price_30', 'atr_14', 'attempt', 'ref_close', 'duration_minutes', 'session', 'eval_f1', 'eval_accuracy', 'eval_precision', 'eval_recall', 'n_total_seen', 'n_window_obs', 'session_code']
📦 Creating TradeSetup with:
  entry_time: 2020-01-02 13:00:00+00:00
  atr_14: nan, ma_14: nan, min_price_30: nan, max_price_30: nan
  context keys: ['ma_14', 'min_price_30', 'max_price_30', 'atr_14', 'attempt', 'ref_close', 'duration_minutes', 'session', 'eval_f1', 'eval_

/Users/manuelheeren/opt/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")
2025-06-17 11:28:25,252 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-17 11:29:09,152 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing



🔍 Entry Time Check (from shared_signals dict):
Type: <class 'dict'>
Keys in shared_signals: ['asian', 'london', 'us']
Type of entry_time: <class 'pandas._libs.tslibs.timestamps.Timestamp'>


2025-06-17 11:29:09,393 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing
2025-06-17 11:29:09,444 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing



📊 Baseline features in .data:
['Gmt time', 'open', 'high', 'low', 'close', 'volume', 'date', 'time', 'asian_session', 'london_session', 'us_session', 'prev_close', 'daily_high', 'daily_low', 'daily_close', 'true_range', 'atr_14', 'ma_14', 'ma_30', 'ma_100', 'day_of_week', 'week_number', 'max_price_14', 'min_price_14', 'max_price_30', 'min_price_30', 'max_price_100', 'min_price_100', 'drawdown_static', 'drawdown_30']

📊 Filtered features in .data:
['Gmt time', 'open', 'high', 'low', 'close', 'volume', 'date', 'time', 'asian_session', 'london_session', 'us_session', 'prev_close', 'daily_high', 'daily_low', 'daily_close', 'true_range', 'atr_14', 'ma_14', 'ma_30', 'ma_100', 'day_of_week', 'week_number', 'max_price_14', 'min_price_14', 'max_price_30', 'min_price_30', 'max_price_100', 'min_price_100', 'drawdown_static', 'drawdown_30']

🕒 Baseline index: timestamp datetime64[ns, UTC]
🕒 Filtered index: timestamp datetime64[ns, UTC]
📦 Creating TradeSetup with:
  entry_time: 2023-06-08 00:00:00

2025-06-17 11:29:35,151 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_fixed.csv
2025-06-17 11:29:35,220 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_fixed.txt
2025-06-17 11:29:35,498 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_fixed.csv



=== XAUUSD Backtest Results ===
Bet Sizing Strategy: fixedfractionalbetsizing

Period: 2023-06-08 to 2024-11-22

ASIAN Session Performance:
Initial Capital: $100,000.00
Final Capital: $100,513.65
Total PnL: $513.65
Return: 0.51%
Win Rate: 49.54% (214W/218L)
Max Drawdown: 1.18%
Sharpe Ratio: 0.33
Skewness: 0.014
Excess Kurtosis: -0.894

Attempt Analysis:

  Attempt 1:
  Trades: 382 (88.4% of session trades)
  PnL: $904.53
  Win Rate: 50.00% (191W/191L)
  Average Win: $54.49
  Average Loss: $-50.54

  Attempt 2:
  Trades: 45 (10.4% of session trades)
  PnL: $-46.10
  Win Rate: 51.11% (23W/22L)
  Average Win: $34.61
  Average Loss: $-38.27

  Attempt 3:
  Trades: 5 (1.2% of session trades)
  PnL: $-344.78
  Win Rate: 0.00% (0W/5L)
  Average Win: $0.00
  Average Loss: $-68.96

LONDON Session Performance:
Initial Capital: $100,000.00
Final Capital: $103,825.56
Total PnL: $3,825.56
Return: 3.83%
Win Rate: 50.19% (264W/262L)
Max Drawdown: 1.13%
Sharpe Ratio: 1.28
Skewness: 0.083
Excess Kurto

2025-06-17 11:29:35,559 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_fixed.txt



=== XAUUSD Backtest Results ===
Bet Sizing Strategy: fixedfractionalbetsizing

Period: 2023-06-08 to 2024-11-20

ASIAN Session Performance:
Initial Capital: $100,000.00
Final Capital: $98,671.13
Total PnL: $-1,328.87
Return: -1.33%
Win Rate: 44.65% (96W/119L)
Max Drawdown: 1.62%
Sharpe Ratio: -1.59
Skewness: 0.131
Excess Kurtosis: -0.794

Attempt Analysis:

  Attempt 1:
  Trades: 186 (86.5% of session trades)
  PnL: $-1,284.91
  Win Rate: 42.47% (79W/107L)
  Average Win: $52.07
  Average Loss: $-51.42

  Attempt 2:
  Trades: 26 (12.1% of session trades)
  PnL: $98.11
  Win Rate: 65.38% (17W/9L)
  Average Win: $29.61
  Average Loss: $-45.02

  Attempt 3:
  Trades: 3 (1.4% of session trades)
  PnL: $-142.07
  Win Rate: 0.00% (0W/3L)
  Average Win: $0.00
  Average Loss: $-47.36

LONDON Session Performance:
Initial Capital: $100,000.00
Final Capital: $101,200.06
Total PnL: $1,200.06
Return: 1.20%
Win Rate: 49.17% (89W/92L)
Max Drawdown: 0.69%
Sharpe Ratio: 1.07
Skewness: 0.136
Excess Kurt